In [1]:
import requests
from bs4 import BeautifulSoup
import pandas as pd, numpy as np
import html5lib
import time
import pickle
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
pd.set_option("display.precision", 2)
# pd.set_option("display.freeze_panes", 1)
from datetime import datetime
now = datetime.now()
print(now.strftime('%d/%m/%Y %H:%M:%S'))
import utilities
import pyspark
# spark = SparkSession.builder.getOrCreate()
# check scikit-learn version
import sklearn
print(sklearn.__version__)

from sklearn.datasets import make_classification
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedStratifiedKFold
from sklearn.ensemble import GradientBoostingClassifier

16/12/2023 11:25:11
1.1.2


In [4]:
# Read the pickled object from the file
df = pd.read_pickle("all_matches_2324.pkl")
# df2 = pd.read_pickle("all_matches_2223.pkl")
df.head(3)
# df2.head(3)

,date,time,comp,round,day,venue,result,gf,ga,opponent,xg,xga,poss,attendance,captain,formation,referee,match report,notes,season,campionato,team,shoot_sh,shoot_sot,shoot_dist,shoot_fk,shoot_pk,shoot_pkatt,poss_touches,poss_def pen,poss_def 3rd,poss_mid 3rd,poss_att 3rd,poss_att pen,poss_live,poss_att,poss_succ,poss_succ%,poss_tkld,poss_tkld%,poss_carries,poss_totdist,poss_prgdist,poss_prgc,poss_1/3,poss_cpa,poss_mis,poss_dis,poss_rec,poss_prgr,pass_total_cmp,pass_total_att,pass_total_cmp%,pass_total_totdist,pass_total_prgdist,pass_short_cmp,pass_short_att,pass_short_cmp%,pass_medium_cmp,pass_medium_att,pass_medium_cmp%,pass_long_cmp,pass_long_att,pass_long_cmp%,pass_ast,pass_xag,pass_xa,pass_kp,pass_1/3,pass_ppa,pass_crspa,pass_prgp,round_1
0,2023-08-19,20:45,Serie A,Matchweek 1,Sat,Home,W,2.0,0.0,Monza,2.7,0.8,47.0,NaN,Lautaro Martínez,3-5-2,Andrea Colombo,Match Report,NaN,2023-2024,11/Serie-A-Stats,Internazionale,22.0,3.0,17.2,1.0,0,0,604.0,58.0,151.0,266.0,195.0,35.0,604.0,17.0,6.0,35.3,8.0,47.1,377.0,1904.0,1090.0,23.0,12.0,7.0,5.0,5.0,441.0,45.0,448.0,509.0,88.0,7722.0,2716.0,226.0,245.0,92.2,144.0,159.0,90.6,58.0,78.0,74.4,2,2.4,1.9,19.0,41.0,9.0,3.0,46.0,1
1,2023-08-28,20:45,Serie A,Matchweek 2,Mon,Away,W,2.0,0.0,Cagliari,0.9,0.6,67.0,NaN,Lautaro Martínez,3-5-2,Michael Fabbri,Match Report,NaN,2023-2024,11/Serie-A-Stats,Internazionale,17.0,3.0,15.9,0.0,0,0,845.0,62.0,257.0,389.0,205.0,40.0,845.0,9.0,2.0,22.2,6.0,66.7,592.0,2748.0,1448.0,26.0,26.0,6.0,13.0,6.0,614.0,64.0,616.0,730.0,84.4,10857.0,3487.0,257.0,277.0,92.8,277.0,311.0,89.1,55.0,98.0,56.1,2,0.6,1.3,10.0,49.0,18.0,5.0,64.0,2
2,2023-09-03,18:30,Serie A,Matchweek 3,Sun,Home,W,4.0,0.0,Fiorentina,3.7,0.4,40.0,NaN,Lautaro Martínez,3-5-2,Matteo Marchetti,Match Report,NaN,2023-2024,11/Serie-A-Stats,Internazionale,20.0,10.0,14.5,1.0,1,1,468.0,52.0,149.0,223.0,102.0,25.0,467.0,5.0,2.0,40.0,1.0,20.0,273.0,1336.0,687.0,13.0,7.0,4.0,12.0,2.0,299.0,36.0,303.0,371.0,81.7,5579.0,2089.0,130.0,148.0,87.8,130.0,149.0,87.2,33.0,53.0,62.3,3,2.5,2.2,15.0,19.0,11.0,4.0,36.0,3


In [8]:
df.agg(max('date'))

AttributeError: 't' is not a valid function for 'DataFrame' object

In [6]:
col = [i for i in df.columns if 'pass'  in i ]
gol = [i for i in df.columns if 'team'  in i or  'gf' in i ]
df[gol + col].head()

,gf,team,pass_total_cmp,pass_total_att,pass_total_cmp%,pass_total_totdist,pass_total_prgdist,pass_short_cmp,pass_short_att,pass_short_cmp%,pass_medium_cmp,pass_medium_att,pass_medium_cmp%,pass_long_cmp,pass_long_att,pass_long_cmp%,pass_ast,pass_xag,pass_xa,pass_kp,pass_1/3,pass_ppa,pass_crspa,pass_prgp
0,2.0,Internazionale,448.0,509.0,88.0,7722.0,2716.0,226.0,245.0,92.2,144.0,159.0,90.6,58.0,78.0,74.4,2,2.4,1.9,19.0,41.0,9.0,3.0,46.0
1,2.0,Internazionale,616.0,730.0,84.4,10857.0,3487.0,257.0,277.0,92.8,277.0,311.0,89.1,55.0,98.0,56.1,2,0.6,1.3,10.0,49.0,18.0,5.0,64.0
2,4.0,Internazionale,303.0,371.0,81.7,5579.0,2089.0,130.0,148.0,87.8,130.0,149.0,87.2,33.0,53.0,62.3,3,2.5,2.2,15.0,19.0,11.0,4.0,36.0
3,5.0,Internazionale,305.0,377.0,80.9,5601.0,2314.0,128.0,143.0,89.5,120.0,136.0,88.2,40.0,65.0,61.5,4,1.7,1.3,10.0,24.0,6.0,3.0,27.0
4,1.0,Internazionale,394.0,494.0,79.8,7212.0,2337.0,158.0,181.0,87.3,195.0,221.0,88.2,37.0,70.0,52.9,1,0.5,0.5,6.0,27.0,5.0,2.0,23.0


In [18]:
col

['pass_total_cmp',
 'pass_total_att',
 'pass_total_cmp%',
 'pass_total_totdist',
 'pass_total_prgdist',
 'pass_short_cmp',
 'pass_short_att',
 'pass_short_cmp%',
 'pass_medium_cmp',
 'pass_medium_att',
 'pass_medium_cmp%',
 'pass_long_cmp',
 'pass_long_att',
 'pass_long_cmp%',
 'pass_ast',
 'pass_xag',
 'pass_xa',
 'pass_kp',
 'pass_1/3',
 'pass_ppa',
 'pass_crspa',
 'pass_prgp']